In [1]:
# import lib

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
#import pypyodbc 
## 加载包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib.font_manager import FontProperties

import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error,mean_squared_error


%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'


import sys
import os

import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

import itertools
import warnings

# Display and Plotting
import matplotlib.pylab as plt
import seaborn as sns

pd.set_option('display.float_format', lambda x: '%.5f' % x) # pandas
np.set_printoptions(precision=5, suppress=True) # numpy

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# seaborn plotting style
sns.set(style='ticks', context='poster')
from datetime import timedelta
import pandas_datareader.data as web

In [3]:
def tsplot(y, lags=None, title='', figsize=(14, 8)):
    fig = plt.figure(figsize=figsize)
    layout = (2, 2)
    ts_ax   = plt.subplot2grid(layout, (0, 0))
    hist_ax = plt.subplot2grid(layout, (0, 1))
    acf_ax  = plt.subplot2grid(layout, (1, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 1))
    
    y.plot(ax=ts_ax)
    ts_ax.set_title(title)
    y.plot(ax=hist_ax, kind='hist', bins=25)
    hist_ax.set_title('Histogram')
    smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
    smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
    [ax.set_xlim(0) for ax in [acf_ax, pacf_ax]]
    sns.despine()
    fig.tight_layout()
    return ts_ax, acf_ax, pacf_ax
from statsmodels.tsa.stattools import adfuller
def test_stationarity(ts):
    ##dfoutput_x[1]<0.05 if true stationary 
    dftest = sm.tsa.adfuller(ts,autolag='BIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    #print('independent stationary : %s'%(dfoutput_x[1]<0.05))
    #print(dfoutput)
    return dfoutput[1]<0.05
    print('dependent stationary : %s'%(dfoutput[1]<0.05))
    
def convert_dataframe_to_list(df):
    lst=[]
    for i in range(df.shape[0]):
        lst.append(df.iloc[i]['SYMBOL'])
    return lst

# Time series module 

In [4]:
def forecast_performance_next_5_days_with_validation(start_date, end_date, ticker):
    try:
        print(ticker)
        pred_days = 5
        df = pd.concat([web.DataReader(ticker,'yahoo', start_date, end_date) ]).reset_index()
        data = df.iloc[:,df.columns.isin(['Date','Adj Close'])]
        data['Return'] = data['Adj Close'].diff().replace(np.nan, 0)
        data['Stationary'] = test_stationarity(data['Return'])
        fdata = pd.DataFrame(data.Return)

        obs = data.iloc[-pred_days:]
        fdata = data.iloc[:-pred_days]
        fdata = pd.DataFrame(fdata.Return)
        # forcast for next 5 days 
        dataset_train= fdata

        # Feature Scaling
        from sklearn.preprocessing import MinMaxScaler
        sc = MinMaxScaler(feature_range = (0, 1))
        training_set_scaled = sc.fit_transform(dataset_train)

        n = dataset_train.shape[0]


        # Creating a data structure with 60 timesteps and 1 output
        ######### slicing window 
        X_train = []
        y_train = []
        for i in range(60, n):
            X_train.append(training_set_scaled[i-60:i, 0])
            y_train.append(training_set_scaled[i, 0])
        X_train, y_train = np.array(X_train), np.array(y_train)

        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


        # Part 2 - Building the RNN

        # Importing the Keras libraries and packages
        from keras.models import Sequential
        from keras.layers import Dense
        from keras.layers import LSTM
        from keras.layers import Dropout

        # Initialising the RNN
        regressor = Sequential()

        # Adding the first LSTM layer and some Dropout regularisation
        regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
        regressor.add(Dropout(0.2))

        # Adding a second LSTM layer and some Dropout regularisation
        regressor.add(LSTM(units = 50, return_sequences = True))
        regressor.add(Dropout(0.2))

        # Adding a third LSTM layer and some Dropout regularisation
        regressor.add(LSTM(units = 50, return_sequences = True))
        regressor.add(Dropout(0.2))

        # Adding a fourth LSTM layer and some Dropout regularisation
        regressor.add(LSTM(units = 50))
        regressor.add(Dropout(0.2))

        # Adding the output layer
        regressor.add(Dense(units = 1))

        # Compiling the RNN
        regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

        # Fitting the RNN to the Training set
        regressor.fit(X_train, y_train, epochs = 3, batch_size = 32)

        inputs = dataset_train[- 60:].values
        inputs = inputs.reshape(-1,1)
        inputs = sc.transform(inputs)
        hist = []
        hist = inputs
        res = np.array([])
        for i in range(pred_days):
            pred = predict_single_stock(hist[i:],regressor)


            res = np.append(res,pred)
            hist = np.append(hist , pred)
        res = pd.DataFrame(res)
        res.columns = ['pred_return']


        #real_stock_price = obs.reset_index(drop = True)
        real_stock_price = list(obs.Return.values)
        real_stock_price = pd.DataFrame(real_stock_price)
        #real_stock_price['datetime'] = obs['Date'].values
        real_stock_price.columns = ['real_return']
        # predict pipeline 
        res = pd.concat([res,real_stock_price],axis = 1)
        res['Diff'] = res['pred_return'].astype('float') - res['real_return'].astype('float')
        msqrt = np.mean((res['pred_return'].astype('float') - res['real_return'].astype('float'))**2)
        res['ticker'] = ticker
        res['rmse'] = msqrt
    except:
        res = pd.DataFrame()
        pass 
        print("no data for: %s"%ticker)
    return res


def predict_single_stock(inputs,regressor):
    X_test = []

    X_test.append(inputs)
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = regressor.predict(X_test)
    return  predicted_stock_price



def prediction_pipeline(dataset_train):
    from sklearn.preprocessing import MinMaxScaler
    sc = MinMaxScaler(feature_range = (0, 1))
    
    inputs = dataset_train[- 60:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    hist = []
    hist = inputs
    res = np.array([])
    for i in range(pred_days):
        pred = predict_single_stock(hist[i:])
        res = np.append(res,pred)
        hist = np.append(hist , pred)
    res = pd.DataFrame(res)
    res.columns = ['pred_return']


    #real_stock_price = obs.reset_index(drop = True)
    real_stock_price = list(obs.Return.values)
    real_stock_price = pd.DataFrame(real_stock_price)
    real_stock_price.columns = ['real_return']
    # predict pipeline 
    res = pd.concat([res,real_stock_price],axis = 1)
    res['Diff'] = res['pred_return'].astype('float') - res['real_return'].astype('float')
    msqrt = np.mean((res['pred_return'].astype('float') - res['real_return'].astype('float'))**2)
    res['ticker'] = ticker
    res['rmse'] = msqrt
    return res 

def eveluate_spy500(start_date, end_date, tickers):
    res = pd.DataFrame()

    for ticker in tickers:
        temp_res  =  forecast_performance_next_5_days_with_validation(start_date, end_date, ticker)
        res = pd.concat([res, temp_res],axis= 0 )
    return res

# Portfolio Construction 

In [5]:
def initialize_weights(stocks):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    return weights
def calculate_portfolio_return(returns, weights):
    # ammual 
    portfolio_return = np.sum(returns.mean()*weights)*252
    return portfolio_return
    print("Expected portfolio return:", portfolio_return)#expected portfolio variance
def calculate_portfolio_variance(returns, weights):
    portfolio_variance = np.sqrt(np.dot(weights.T, np.dot(np.cov(returns)*252,weights)))
    return  portfolio_variance
    print("Expected variance:", portfolio_variance)
    
def generate_portfolios(weights, returns,stocks):

	preturns = []
	pvariances = []

	#Monte-Carlo simulation: we generate several random weights -> so random portfolios !!!
	for i in range(100000):
		weights = np.random.random(len(stocks))
		weights/=np.sum(weights)
		preturns.append(np.sum(returns.mean()*weights)*252)
		pvariances.append(np.sqrt(np.dot(weights.T,np.dot(np.cov(returns)*252,weights))))
	
	preturns = np.array(preturns)
	pvariances = np.array(pvariances)
	return preturns,pvariances

# what are the constraints? The sum of weights = 1 !!!  f(x)=0 this is the function to minimize
def optimize_portfolio(weights,returns,stocks):
    import scipy.optimize as optimization
    constraints = ({'type':'eq','fun': lambda x: np.sum(x)-1}) #the sum of weights is 1
    bounds = tuple((0,1) for x in range(len(stocks))) 
    #the weights can be 1 at most: 1 when 100% of money is invested into a single stock
    optimum=optimization.minimize(fun=min_func_sharpe,
                                  x0=weights,args=returns,
                                  method='SLSQP',
                                  bounds=bounds,
                                  constraints=constraints) 
    return optimum
def	min_func_sharpe(weights,returns):
	return	-statistics(weights,returns)[2]
def statistics(weights, returns):
	portfolio_return=np.sum(returns.mean()*weights)*252
	portfolio_volatility=np.sqrt(np.dot(weights.T,np.dot(np.cov(returns)*252,weights)))
	return np.array([portfolio_return,portfolio_volatility,portfolio_return/portfolio_volatility])

def show_optimal_portfolio(optimum, returns, preturns, pvariances):
	plt.figure(figsize=(10,6))
	plt.scatter(pvariances,preturns,c=preturns/pvariances,marker='o')
	plt.grid(True)
	plt.xlabel('Expected Volatility')
	plt.ylabel('Expected Return')
	plt.colorbar(label='Sharpe Ratio')
	plt.plot(statistics(optimum['x'],returns)[1],statistics(optimum['x'],returns)[0],'g*',markersize=20.0)
	plt.show()
def print_optimal_portfolio(optimum, returns):
	print("Optimal weights:", optimum['x'].round(3))
	print("Expected return, volatility and Sharpe ratio:", statistics(optimum['x'].round(3),returns))
    
def plot_portfolios(returns, variances):
	plt.figure(figsize=(10,6))
	plt.scatter(variances,returns,c=returns/variances,marker='o')
	plt.grid(True)
	plt.xlabel('Expected Volatility')
	plt.ylabel('Expected Return')
	plt.colorbar(label='Sharpe Ratio')
	plt.show()
    
def get_portfolio_universe(res):
    print(res)
    summary2 = res.groupby('ticker').std().reset_index()
    summary = res.groupby('ticker').sum().reset_index()
    summary['std'] = summary2['pred_return']
    summary['return/std'] = summary['pred_return']/summary2['pred_return']
    universe = summary.nlargest(5,'return/std')
    return universe

def p_l_calculate(universe,source,optimum):
   # pstat = pd.DataFrame()
    #pstat['source'] = source
    pstat = pd.DataFrame(statistics(optimum['x'].round(3),universe[source]))
    pstat = pstat.T
    pstat.columns = ['annual_return','annual_vol','annual_sharpe_ratio']
    return pstat


def construct_portfolio(universe):
    # intitialize weight 
    weights=initialize_weights(universe)
    # calculate predict protfolio return 
    port_return = calculate_portfolio_return(universe['pred_return'], weights)
    # calculate predict portfolio standard deviation 
    port_var = calculate_portfolio_variance(universe['pred_return'], weights)
    # calculate portfolios return 
    preturns,pvariances=generate_portfolios(weights, universe['pred_return'],universe)
    # get optimal weight 
    optimum = optimize_portfolio(weights,universe['pred_return'],universe)
    #optimum['x'] 
    pred_return_5days = np.sum(optimum['x'] * universe['pred_return'])
    real_return_5days = np.sum(optimum['x'] * universe['real_return'])


    # pl summary 
    pred_summary = p_l_calculate(universe,'pred_return',optimum)
    pred_summary['5days_return'] = pred_return_5days
    pred_summary['source'] = 'Prediction'

    real_summary = p_l_calculate(universe,'real_return',optimum)
    real_summary['5days_return'] = real_return_5days
    real_summary['source'] = 'real'

    P_L_summary = pd.concat([pred_summary ,real_summary],axis = 0)

    #portfolio summary 
    equity = pd.DataFrame(universe['ticker']).reset_index(drop = True)
    weight = pd.DataFrame(optimum['x'])

    portfolio_equity = pd.concat([equity,weight],axis = 1)
    portfolio_equity.columns = ['ticker','weight']
    return P_L_summary,portfolio_equity

In [6]:
def one_period_implement(start_date, end_date, tickers):
    # forecast return 
    res =  eveluate_spy500(start_date, end_date, tickers)
    # define good performance equity 
    universe = get_portfolio_universe(res)
    # portfolio construction 
    P_L_summary,portfolio_equity = construct_portfolio(universe)
    return res,universe, P_L_summary,portfolio_equity 

In [7]:
def get_date_index(numdays,interval):
    from dateutil.relativedelta import relativedelta
    import datetime 
    base = datetime.datetime(2018, 1, 11, 0, 0, 0, 0)

    #base = datetime.datetime.today() - datetime.timedelta(days=6)
    date_list = [base - datetime.timedelta(days=x) for x in range(0, numdays*7,7)]
    end_list = pd.to_datetime(date_list).strftime('%Y-%m-%d')
    end_list = pd.DataFrame(end_list)
    start_list = [base - datetime.timedelta(days=x) - relativedelta(years=2) for x in range(0, numdays*7,7)]
    start_list = pd.to_datetime(start_list).strftime('%Y-%m-%d')
    start_list = pd.DataFrame(start_list)
    date_ix = pd.concat([start_list,end_list],axis = 1)
    date_ix.columns = ['start_date','end_date']
    date_ix = date_ix.sort_values(by = 'end_date').reset_index(drop = True)
    return date_ix

In [8]:
def implement_lstm_mc_mv_algo(date_ix,tickers):
    res_all = pd.DataFrame()
    universe_all = pd.DataFrame()
    pl_all = pd.DataFrame()
    portfolio_equity_all = pd.DataFrame()
    for i in range(len(date_ix)):
        start_date  =  date_ix.iloc[i]['start_date']
        end_date    =  date_ix.iloc[i]['end_date']
        print(end_date)
        res,universe, P_L_summary,portfolio_equity = one_period_implement( start_date , end_date, tickers)
        # add time stamp 
        res['DATE'] = end_date
        universe['DATE'] = end_date
        P_L_summary['DATE'] = end_date
        portfolio_equity['DATE'] = end_date

        # aggregate result 
        res_all = pd.concat([res_all,res],axis = 0)
        universe_all = pd.concat([universe_all,universe],axis = 0)
        pl_all = pd.concat([pl_all,P_L_summary],axis = 0)
        portfolio_equity_all = pd.concat([portfolio_equity_all,portfolio_equity],axis = 0)   
    return res_all,universe_all,pl_all,portfolio_equity_all

# Implement 

In [9]:
import datetime 
today = datetime.datetime.now()
prev = today - timedelta(days = 365)
prev  = prev .strftime('%Y-%m-%d')
today = today.strftime('%Y-%m-%d')

In [10]:
#start_date = '2016-1-1'
#end_date = '2018-1-1'

In [11]:
# analysis universe 
sym = pd.read_csv('SYM.csv')
sym.columns = ['SYMBOL']
tickers = sym.iloc[:100]
tickers['SYMBOL'] = tickers['SYMBOL'].str.replace("'","")
tickers['SYMBOL'] = tickers['SYMBOL'].str.strip()
tickers =  convert_dataframe_to_list(tickers)

C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
# single period simulation 
#res,universe, P_L_summary,portfolio_equity   = one_period_implement(start_date, end_date, tickers)

In [13]:
rebalance_interval = 7 
invest_period = 2
date_ix = get_date_index(invest_period,rebalance_interval)

In [14]:
res_all,universe_all,pl_all,portfolio_equity_all = implement_lstm_mc_mv_algo(date_ix,tickers)

2018-01-04
ABT


C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
442/442 [==============================] - 3s 7ms/step - loss: 0.1495
Epoch 2/3
442/442 [==============================] - 1s 2ms/step - loss: 0.0278
Epoch 3/3
442/442 [==============================] - 1s 2ms/step - loss: 0.0156A: 0s - loss: 0.015
ABBV


C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Epoch 1/3
442/442 [==============================] - 3s 7ms/step - loss: 0.0751
Epoch 2/3
442/442 [==============================] - 1s 2ms/step - loss: 0.0183
Epoch 3/3
442/442 [==============================] - 1s 2ms/step - loss: 0.0143
ACN


...

Epoch 1/3
442/442 [==============================] - 425s 962ms/step - loss: 0.1175
Epoch 2/3
442/442 [==============================] - 12s 27ms/step - loss: 0.0223
Epoch 3/3
442/442 [==============================] - 12s 28ms/step - loss: 0.0201
XEC


C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Epoch 1/3
442/442 [==============================] - 432s 978ms/step - loss: 0.0465
Epoch 2/3
442/442 [==============================] - 12s 26ms/step - loss: 0.0151
Epoch 3/3
442/442 [==============================] - 12s 26ms/step - loss: 0.0124
CINF


C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\AQi\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Epoch 1/3
442/442 [==============================] - 438s 991ms/step - loss: 0.1037
Epoch 2/3
442/442 [==============================] - 12s 28ms/step - loss: 0.0203
Epoch 3/3
442/442 [==============================] - 12s 28ms/step - loss: 0.0143
    pred_return  real_return     Diff ticker    rmse
0       0.71165     -0.16549  0.87714    ABT 0.51440
1       0.71275      0.09735  0.61541    ABT 0.51440
2       0.71374     -0.08762  0.80135    ABT 0.51440
3       0.71459      0.39125  0.32334    ABT 0.51440
4       0.71531     -0.10759  0.82291    ABT 0.51440
0       0.44716     -1.52016  1.96732   ABBV 1.04235
1       0.44764      0.70379 -0.25615   ABBV 1.04235
2       0.44835     -0.51609  0.96444   ABBV 1.04235
3       0.44917      0.27407  0.17510   ABBV 1.04235
4       0.45002      1.01125 -0.56124   ABBV 1.04235
0       0.58200      1.22749 -0.64550    ACN 0.39804
1       0.58239      0.51633  0.06606    ACN 0.39804
2       0.58302     -0.54555  1.12857    ACN 0.39804
3       0.

In [15]:
universe_all

,ticker,pred_return,real_return,Diff,rmse,std,return/std,DATE
15,AES,2.74054,0.10393,2.63661,1.39771,0.00010,26589.83609,2018-01-04
54,BMY,4.14605,0.50388,3.64217,3.42385,0.00024,17492.68535,2018-01-04
77,HSIC,3.15017,2.17255,0.97762,2.17683,0.00021,15297.75474,2018-01-04
62,CCL,3.01520,-0.17308,3.18828,2.47426,0.00020,14905.37704,2018-01-04
17,AGN,3.53400,7.49965,-3.96565,38.85779,0.00024,14731.75776,2018-01-04
58,CAH,3.65367,7.89430,-4.24063,5.00371,0.00006,59692.13815,2018-01-11
21,AIZ,3.07489,-5.74611,8.82100,18.80776,0.00006,49960.03935,2018-01-11
25,ALXN,2.96739,-0.09000,3.05739,17.09594,0.00010,29245.56704,2018-01-11
62,CCL,3.05246,3.25942,-0.20696,1.74451,0.00017,17515.24061,2018-01-11
10,ADM,2.89578,0.30814,2.58764,1.82749,0.00018,15689.46250,2018-01-11


In [16]:
portfolio_equity_all

,ticker,weight,DATE
0,AES,0.20000,2018-01-04
1,BMY,0.20000,2018-01-04
2,HSIC,0.20000,2018-01-04
3,CCL,0.20000,2018-01-04
4,AGN,0.20000,2018-01-04
0,CAH,0.20001,2018-01-11
1,AIZ,0.19999,2018-01-11
2,ALXN,0.20000,2018-01-11
3,CCL,0.20000,2018-01-11
4,ADM,0.20000,2018-01-11


In [17]:
pl_all

,annual_return,annual_vol,annual_sharpe_ratio,5days_return,source,DATE
0,835.93265,3.86510,216.27721,3.31719,Prediction,2018-01-04
0,509.38907,22.68149,22.45836,2.02140,real,2018-01-04
0,788.46696,2.14329,367.87616,3.12884,Prediction,2018-01-11
0,283.53749,35.44939,7.99838,1.12527,real,2018-01-11


In [18]:
res_all

,pred_return,real_return,Diff,ticker,rmse,DATE
0,0.74563,-0.37966,1.12529,ABT,0.71411,2018-01-04
1,0.74694,1.67440,-0.92746,ABT,0.71411,2018-01-04
2,0.74806,0.12656,0.62150,ABT,0.71411,2018-01-04
3,0.74902,-0.09735,0.84637,ABT,0.71411,2018-01-04
4,0.74985,0.16549,0.58436,ABT,0.71411,2018-01-04
0,0.44691,-1.01344,1.46035,ABBV,1.35953,2018-01-04
1,0.44681,1.59523,-1.14842,ABBV,1.35953,2018-01-04
2,0.44657,1.44508,-0.99850,ABBV,1.35953,2018-01-04
3,0.44624,-0.53487,0.98111,ABBV,1.35953,2018-01-04
4,0.44584,1.62337,-1.17753,ABBV,1.35953,2018-01-04


In [19]:
#universe_all.to_csv("2019-4-26_buyin_portfolio.csv")

In [20]:
#portfolio_equity_all.to_csv("2019-4-26_portfolio.csv")

In [21]:
#pl_all.to_csv("2019-4-26_highest_return.csv")

In [22]:
#res_all.to_csv("2019-4-26_forcasting.csv")

In [23]:
#portfolio_equity

In [24]:
# Portfolio Return 